In [1]:
!pip install -q transformers sentencepiece torch-summary torchmetrics

In [2]:
import pandas as pd
import numpy as np

import os, re

from transformers import T5Tokenizer, T5ForConditionalGeneration
from torchmetrics.text import BLEUScore
from torchsummary import summary

from sklearn.model_selection import train_test_split

In [3]:
DATASET_DEFAULT = "/kaggle/input/indonesian-informal-to-formal-text/text_translate"
dataset_default = pd.read_excel(
    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset.xlsx"
)

dataset_extra = pd.read_csv(
    f"{DATASET_DEFAULT}/Indonesian cola dataset v3.csv",
    encoding="utf-8"
)

"""dataset_10 = pd.read_csv(
    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-10.csv",
    encoding="utf-8"
)

dataset_20 = pd.read_csv(
    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-20.csv",
    encoding="utf-8"
)

dataset_30 = pd.read_csv(
    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-30.csv",
    encoding="utf-8"
)

dataset_40 = pd.read_csv(
    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-40.csv",
    encoding="utf-8"
)

dataset_50 = pd.read_csv(
    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-50.csv",
    encoding="utf-8"
)"""

/tmp/ipykernel_24/1537688226.py:6: DtypeWarning: Columns (2,3,4,5,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_extra = pd.read_csv(


'dataset_10 = pd.read_csv(\n    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-10.csv",\n    encoding="utf-8"\n)\n\ndataset_20 = pd.read_csv(\n    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-20.csv",\n    encoding="utf-8"\n)\n\ndataset_30 = pd.read_csv(\n    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-30.csv",\n    encoding="utf-8"\n)\n\ndataset_40 = pd.read_csv(\n    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-40.csv",\n    encoding="utf-8"\n)\n\ndataset_50 = pd.read_csv(\n    f"{DATASET_DEFAULT}/Indonesian Augmented Dataset-50.csv",\n    encoding="utf-8"\n)'

# Preprocessing

In [4]:
dataset_extra = dataset_extra[["Kalimat","Formal Sentences"]]
dataset_extra.shape

(179030, 2)

In [5]:
dataset_extra.dropna(inplace=True)

In [6]:
dataset_extra.shape

(2405, 2)

In [7]:
"""dataset_10,
dataset_20,
dataset_30,
dataset_40,
dataset_50,"""
    
dataset = pd.concat([
    dataset_default,
    dataset_extra
], ignore_index=True)

dataset.shape

(21053, 3)

In [8]:
# Drop Duplication
duplication = dataset.duplicated(subset=['Kalimat']).sum()

print(f'Duplicaton Sum {duplication}')

if duplication:
  dataset.drop_duplicates(
      subset=['Kalimat'],
      inplace=True,
      ignore_index=True,
  )

Duplicaton Sum 1730


In [9]:
dataset.shape

(19323, 3)

In [10]:
# Remove _ values
dataset['Kalimat'] = dataset['Kalimat'].astype('str')
dataset['Kalimat'] = dataset['Kalimat'].apply(lambda x:re.sub('_',' ',x))

In [11]:
# Add Prefix Mode
prefix_df = pd.DataFrame({
    'prefix' : ['informal converter'] * dataset.shape[0]
})

# Reset Index
dataset.reset_index(inplace=True)
new_dataset = pd.merge(prefix_df, dataset, left_index=True, right_index=True)
new_dataset.shape

(19323, 5)

In [12]:
# Drop Unused Col
new_dataset.drop(columns=['Unnamed: 0', 'index'], inplace=True)
new_dataset.shape

(19323, 3)

In [13]:
# Format Cols Name
new_dataset.columns = ['prefix', 'input_text', 'target_text']

train_datasets, test_datasets = train_test_split(
    new_dataset, 
    test_size=0.2, 
    shuffle=True, 
    random_state=10000
)
train_datasets, val_datasets = train_test_split(
    train_datasets, 
    test_size=0.2, 
    shuffle=True, 
    random_state=10000
)

print(train_datasets.shape)
print(val_datasets.shape)
print(test_datasets.shape)

(12366, 3)
(3092, 3)
(3865, 3)


In [14]:
val_datasets.tail()

,prefix,input_text,target_text
4066,informal converter,"['benjamin radford tahu, pada masa awal manusi...","Benjamin Radford memaparkan, pada masa lampau ..."
16408,informal converter,"Pertama, dia meminta para pemain lokal untuk m...","Pertama, dia meminta para pemain lokal untuk m..."
3660,informal converter,['krisis etnis rohingya yang terjadi di myanma...,Krisis etnis Rohingya yang terjadi di Myanmar ...
7595,informal converter,['Mereka menawarkan melalui Midia mereka misal...,Mereka menawarkan melalui media mereka misalny...
17830,informal converter,"QnC Jelly Gamat, yang dimana teripang Emas lau...","QnC Jelly Gamat, Teripang Emas laut sebagai ba..."


# Dataloader

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

class DatasetGenerator(Dataset):
    def __init__(self, data, tokenizer, text_max_token_len, max_token_target_len = 15, train=False):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.max_token_target_len = max_token_target_len
        self.train = train
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data = self.data.iloc[index]
        
        inputs = str(' '.join(str(data['input_text']).split()))
        targets = str(' '.join(str(data['target_text']).split()))
        
                      
        text_encoding = self.tokenizer(
            [inputs],
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        target_encoding = tokenizer(
            [targets],
            max_length=self.max_token_target_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        labels = target_encoding['input_ids']
        if self.train:
            labels[labels == tokenizer.pad_token_id] = -100

        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=target_encoding['attention_mask'].flatten()
        )

# Modelling - T5
---
* https://wandb.ai/yepster/tpu-t5-base/reports/Adafactor-learning-rate-0-005-seems-best-for-t5-base-training--VmlldzoxNTgyODIw

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
"""from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name='indonesian-nlp/gpt2-medium-indonesian'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2LMHeadModel.from_pretrained(model_name)"""

from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("panggi/t5-base-indonesian-summarization-cased")
model = T5ForConditionalGeneration.from_pretrained("panggi/t5-base-indonesian-summarization-cased").to(device)

tokenizer_config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [18]:
config = {
    'train':8,
    'val':4,
    'test':1,
    'input_length':128,
    'target_length':128
}

In [19]:
train_generator = DatasetGenerator(
    data=train_datasets, 
    tokenizer=tokenizer,
    text_max_token_len=config['input_length'],
    max_token_target_len=config['target_length']
)

val_generator = DatasetGenerator(
    data=val_datasets, 
    tokenizer=tokenizer,
    text_max_token_len=config['input_length'],
    max_token_target_len=config['target_length']
)

test_generator = DatasetGenerator(
    data=test_datasets, 
    tokenizer=tokenizer,
    text_max_token_len=config['input_length'],
    max_token_target_len=config['target_length']
)

dataloaders = {
    'train':DataLoader(
        train_generator,
        shuffle=True, 
        batch_size=config['train']
    ), 'val':DataLoader(
        val_generator,
        shuffle=False, 
        batch_size=config['val']
    ), 'test':DataLoader(
        test_generator,
        shuffle=True, 
        batch_size=config['test']
    )
} 

In [20]:
phase_list = list(dataloaders.keys())
phase_list

['train', 'val', 'test']

In [21]:
x = next(iter(dataloaders['test']))
x

{'input_ids': tensor([[25594,  1877,    84,  2388,    22,  7536,  2323,  1967,    17,    64,
           2249,  1214,    18,   380,  4360,  2323,    24,    14,   135,  1214,
             17, 26695,   158,    29,  4981,     1,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

# Train Phase

In [22]:
import gc
from torch.cuda.amp import autocast
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm
import time
from collections import defaultdict
import json

# Fungsi loss yang digunakan sekaligus update state loss dan metrics
def calc_loss(pred, metrics):
    loss = pred.loss
    metrics['loss'] += loss.cpu().data.numpy()
    return loss


# Print metric per epoch
def print_metrics(metrics, global_metrics, epoch_samples, phase, epoch):    
    outputs = []
    
    for k in metrics.keys():
        outputs.append("{}: {}".format(k, metrics[k] / epoch_samples))
        global_metrics[phase][k].append(metrics[k] / epoch_samples)
        
        
    global_metrics[phase]['epoch'] = epoch
        

    print("{}: {}".format(phase, ", ".join(outputs)))
    return global_metrics

def train_model(dataloader, model, phase_list, optimizer_lr=1e-3, num_epochs=100, checkpoint_path='/content'):
    # Inisialisasi metrics
    global_metrics = {}
    
    for i in phase_list:
        global_metrics[i] = {
            'epoch':[],
            'loss':[]
        }
    
    # Inisialisasi opt
    optimizer = torch.optim.AdamW(model.parameters(), lr=optimizer_lr)
    
    # Scheduler
    num_training_steps = num_epochs * len(dataloader['train'])
    scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )
    scaler = torch.cuda.amp.GradScaler()
    
    # Inisialisasi loss threshold
    best_loss, best_epoch, best_model, last_epoch = 0, 0, 0, 0
    
    # Resume Checkpoint
    """if os.path.exists(f'{checkpoint_path.split("pth")[0]}_checkpoint.pth'):
        with open(f"{checkpoint_path.split('pth')[0]}json", "r") as f: 
            global_metrics = json.load(f)
            
        last_epoch = len(global_metrics['train']['loss'])
        
        checkpoint = torch.load(
            f'{checkpoint_path.split("pth")[0]}_checkpoint.pth', 
            map_location=torch.device(device)
        )
        
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        model.load_state_dict(checkpoint['model'])
        
        print(f"Load Checkpoint: {checkpoint['epoch']}")"""
    
    for epoch in range(last_epoch, num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in phase_list:
            num_training_steps = len(dataloaders[phase])
            progress_bar = tqdm(range(num_training_steps))
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                
            # Inisialisasi metric dict guna menyimpan hasil pembelajaran
            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs in dataloaders[phase]:
                inputs = {k: v.to(device) for k, v in inputs.items()}
                
                # reset gradient parameter
                model.zero_grad()

                # forward pass
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    with autocast(True):
                        # Generator Training
                        outputs = model(**inputs)
                        loss = calc_loss(outputs, metrics)
                    
                    # backpropagation in training phase
                    if phase == 'train':                       
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()
                        
                # statistics
                epoch_samples += config[phase]
                progress_bar.update()
                
                del inputs, outputs
                gc.collect()

            global_metrics = print_metrics(metrics, global_metrics, epoch_samples, phase, epoch)
            epoch_loss = metrics['loss'] / epoch_samples
            

            if phase == 'train':
              scheduler.step()
              for param_group in optimizer.param_groups:
                  print("LR G", param_group['lr'])

            # fase val utk testing dan saving weights
            if phase =='val':
                if (epoch+1) % 1 == 0 or epoch == 0:
                    inputs = next(iter(dataloaders['test']))
                    inputs = {k: v.to(device) for k, v in inputs.items()}
                    
                    outputs = model.generate(
                        input_ids = inputs['input_ids'],
                        attention_mask = inputs['attention_mask'], 
                        max_length=config['target_length'], 
                        num_beams=2,
                        repetition_penalty=2.5, 
                        length_penalty=1.0, 
                        early_stopping=True
                    )
                    # Label
                    preds = [tokenizer.decode(a, skip_special_tokens=True, clean_up_tokenization_spaces=True) 
                             for a in outputs
                            ]
                    target = [tokenizer.decode(a, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                              for a in inputs['labels']
                             ]
                    
                    print("========== Validation ==========")
                    print("Prediction : ", preds)
                    print("Ground Truth : ", target)
                    
                    del inputs, outputs, preds, target
                    gc.collect()
                     
            if (phase == 'val' and epoch == 0):
                best_loss = epoch_loss
                best_model = model.state_dict()
                
            if (phase == 'val' and epoch_loss < best_loss):
                print(f"saving best model to {checkpoint_path} | Epoch: {best_epoch}")
                best_loss = epoch_loss
                best_epoch = epoch
                best_model = model.state_dict()
                        
            # Metrics to json
            with open(f"{checkpoint_path.split('pth')[0]}json", "w") as f: 
                json.dump(global_metrics, f)

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
        checkpoint = { 
            'epoch': epoch,
            'best_epoch':best_epoch,
            'best_model':best_model,
            'model':model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(checkpoint, f'{checkpoint_path.split("pth")[0]}_checkpoint.pth')

    print('Best val loss: {:4f}'.format(best_loss), 'Best Epoch:', best_epoch)

In [23]:
LR = 2e-4
EPOCHS = 20

train_model(
    dataloader=dataloaders, 
    model=model, 
    phase_list=phase_list[:-1], 
    optimizer_lr=LR, 
    num_epochs=EPOCHS, 
    checkpoint_path='/kaggle/working/T5TextStyleTransfer.pth'
)

Epoch 1/20
----------


  0%|          | 0/1546 [00:00<?, ?it/s]

train: loss: 0.030119016337031342
LR G 0.00019999353169469598


  0%|          | 0/773 [00:00<?, ?it/s]

val: loss: 0.042408650253633134
========== Validation ==========
Prediction :  ['Mereka yang Handy Talky satu rombongan']
Ground Truth :  ['Mereka yang bawa Handy Talky satu rombongan']
14m 20s
Epoch 2/20
----------


...

val: loss: 0.02521295413717671
========== Validation ==========
Prediction :  ['Caranya, perhatikan perhatikan handuk pada salah stau bahan alami di atas, kemudian diaplikasikan pada area yang terasa nyeri.']
Ground Truth :  ['Caranya adalah dengan menempelkan handuk pada salah stau bahan alami tersebut, kemudian diaplikasikan pada area yang terasa nyeri.']
14m 34s
Epoch 20/20
----------


  0%|          | 0/1546 [00:00<?, ?it/s]

train: loss: 0.0004883602176810842
LR G 0.0001998706338939198


  0%|          | 0/773 [00:00<?, ?it/s]

val: loss: 0.025748321004446843
========== Validation ==========
Prediction :  ['Melihat perubahan besar dari adikku, membuat mereka terkejut.']
Ground Truth :  ['Melihat perubahan besar dari adikku, membuat mereka terkejut.']
14m 50s
Best val loss: 0.024493 Best Epoch: 17
